In [74]:
### Imports

import re
import numpy as np
import pandas as pd

In [75]:
def generate_sequence(list_of_words, sequence_length):
    res = []
    for i in range(len(list_of_words)-sequence_length):
        res.append((list_of_words[i:i+sequence_length],list_of_words[i+sequence_length]))
    return res

def splitter(database, fraction):
    cutter = int(fraction*database.shape[0])
    X_train = database.sample(frac=1).iloc[:cutter]['tokens'].array
    X_test = database.sample(frac=1).iloc[cutter:]['tokens'].array
    y_train = database.sample(frac=1).iloc[:cutter]['label_tokens'].array
    y_test = database.sample(frac=1).iloc[cutter:]['label_tokens'].array
    return X_train, X_test, y_train, y_test

In [76]:
books= []


delims = ' |\n|\ufeff'
sequence_length = 50

for i in range(1,11):
    with open(f'../Data/ABT/{i}.txt', mode="r", encoding="utf-8") as file:
        contents = re.split(delims, file.read())
        books.append([x for x in contents if x != ''])

for ind, book in enumerate(books):
    books[ind] = generate_sequence(book, sequence_length=sequence_length)

master = []

for book in books:
    for sequence in book:
        master.append(sequence)

master_frame = pd.DataFrame(master, columns=['sequence','label'])
master_frame.sample(10)

,sequence,label
89942,"[himself, wrote, down, the, words,, supplied, ...",it
644253,"[it.', They, walked, up, the, steep, hill, out...","falling,"
412340,"[severity, of, grey, rock, and, dark, pine, fo...",stage
223460,"[“A, _very_, singular, incident, indeed,”, he,...",ask
688045,"[whither, he, was, going., The, corridor, was,...",he
708170,"[life, and, future,, and, her, own, as, well--...",of
669511,"[of, successful, ""exchange"", in, New, York, Ci...",good
293898,"[To, make, a, god, of, them, was, to, make, an...",to
485644,"[said, faintly,, and, the, girl, came, up, sof...","mine,"""
89999,"[screened, from, observation,, yet, so, placed...","occurrence,"


In [77]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(master_frame['sequence'])
master_frame['tokens'] = tokenizer.texts_to_sequences(master_frame['sequence'])
master_frame['label_tokens'] = tokenizer.texts_to_sequences(master_frame['label'])

word_lexicon = tokenizer.word_index
word_index = tokenizer.index_word
num_words = len(word_lexicon) + 1
word_counts = tokenizer.word_counts
sorted_counts = dict(sorted(dict(word_counts).items(), key=lambda item: item[1],reverse=True))

master_frame.sample(5)

,sequence,label,tokens,label_tokens
674032,"[only, a, crowd, of, trees, and, shadows, hang...",alive,"[66, 5, 3148, 2, 376, 3, 504, 1865, 838, 14, 5...",[1043]
447941,"[I, paused, in, my, dangerous, descent., Throu...","church,","[11, 692, 6, 27, 1387, 16264, 1523, 1, 386, 64...",[2630]
356059,"[rhododendrons., These, formed, no, continuous...",of,"[23030, 932, 1980, 38, 3619, 2845, 29, 136, 50...",[2]
373798,"[at, the, brakes., ""The, sham, and, emptiness,...",powerfully,"[19, 1, 37396, 292, 5131, 3, 6231, 40, 831, 31...",[10797]
360167,"[miscall, delight, could, touch, him, not, nor...","morning,","[22618, 1651, 51, 302, 22, 21, 362, 9432, 22, ...",[614]


In [78]:
X_train, X_test, y_train, y_test = splitter(master_frame,0.9)

In [79]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X_train)
pca.explained_variance_ratio_

ValueError: setting an array element with a sequence.

In [82]:
np.asarray(X_train)

array([list([1, 22391, 313, 10546, 6212, 202, 20187, 6613, 1, 15775, 33877, 121, 1057, 33878, 33, 4929, 2, 36, 1232, 22392, 3, 2046, 10, 6728, 22, 6, 22393, 246, 8, 100, 522, 3, 164, 10, 1, 22394, 33879, 98, 1047, 33880, 4, 927, 6624, 15, 22395, 3, 6, 9, 272, 36]),
       list([99, 31894, 92, 49, 4341, 12, 1503, 49, 1122, 31895, 516, 31896, 31897, 5597, 7148, 19951, 181, 20853, 325, 241, 241, 241, 241, 241, 18, 1436, 7, 5, 209, 919, 2103, 278, 168, 22, 3, 20, 608, 6, 1, 31898, 34, 7, 21, 159, 26, 5095, 1, 627, 54, 269]),
       list([25, 990, 4, 2584, 147, 1, 815, 2, 1, 54, 2740, 10, 2180, 4, 1, 520, 3, 10443, 37082, 277, 1, 335, 2, 1, 1313, 1, 54, 1903, 452, 20, 9974, 6, 1, 22842, 3, 1, 941, 23129, 9, 851, 2644, 16, 7, 379, 24, 1, 23130, 4, 754, 1]),
       ...,
       list([17, 5, 124, 12, 75, 42, 1, 8550, 87, 8, 203, 3840, 9, 95, 97, 9773, 9, 518, 398, 10831, 6622, 9, 1422, 92, 42, 9, 13924, 2617, 1560, 1, 40365, 8, 3528, 10313, 6, 2277, 2, 1701, 3658, 40366, 40367, 16, 692, 5, 373,